In [1]:
# Change working directory to the project root
import os
os.chdir('..')

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
MATCH_RATE_THRESHOLD = 0.4

In [4]:
CASES = [
    'mfcc',
    'chroma',
    'chroma-keyinv',
    'chroma+mfcc',
    'chroma+mfcc-keyinv',
]

In [5]:
df = pd.read_pickle('data/mix_segmentation.pkl')
df.head()

,case,i_track_prev,i_track_next,track_id_prev,track_id_next,match_rate_prev,match_rate_next,timestamp_prev,timestamp_next,mix_cue_out_time,...,timestamp_beat_prev,timestamp_beat_next,mix_cue_out_beat,mix_cue_in_beat,mix_cue_mid_beat,key_change_prev,key_change_next,wp_prev,wp_next,mix_id
0,chroma+mfcc,0,1,0tNDAyqBm2A,yAFEClw71TM,0.819165,0.112654,0,320,283.88,...,0.0,600.0,534.0,243.0,388.0,0,0,"[[647, 555], [646, 555], [645, 555], [644, 555...","[[648, 316], [647, 316], [646, 316], [645, 316...",cPo-qzbGLqE
1,chroma-keyinv,0,1,0tNDAyqBm2A,yAFEClw71TM,0.813846,0.183642,0,320,283.88,...,0.0,600.0,534.0,345.0,439.0,0,11,"[[647, 554], [646, 554], [645, 554], [644, 554...","[[648, 374], [647, 374], [646, 374], [645, 374...",cPo-qzbGLqE
2,mfcc,0,1,0tNDAyqBm2A,yAFEClw71TM,0.851623,0.123664,0,320,304.06,...,0.0,600.0,571.0,2700.0,1635.0,0,0,"[[647, 576], [646, 576], [645, 576], [644, 576...","[[648, 2784], [647, 2784], [646, 2784], [645, ...",cPo-qzbGLqE
3,chroma+mfcc-keyinv,0,1,0tNDAyqBm2A,yAFEClw71TM,0.819165,0.259259,0,320,283.88,...,0.0,600.0,534.0,344.0,439.0,0,11,"[[647, 555], [646, 555], [645, 555], [644, 555...","[[648, 483], [647, 483], [646, 483], [645, 483...",cPo-qzbGLqE
4,chroma,0,1,0tNDAyqBm2A,yAFEClw71TM,0.813846,0.089506,0,320,283.88,...,0.0,600.0,534.0,243.0,388.0,0,0,"[[647, 554], [646, 554], [645, 554], [644, 554...","[[648, 301], [647, 301], [646, 300], [645, 300...",cPo-qzbGLqE


In [6]:
df.columns

Index(['case', 'i_track_prev', 'i_track_next', 'track_id_prev',
       'track_id_next', 'match_rate_prev', 'match_rate_next', 'timestamp_prev',
       'timestamp_next', 'mix_cue_out_time', 'mix_cue_in_time',
       'mix_cue_mid_time', 'timestamp_beat_prev', 'timestamp_beat_next',
       'mix_cue_out_beat', 'mix_cue_in_beat', 'mix_cue_mid_beat',
       'key_change_prev', 'key_change_next', 'wp_prev', 'wp_next', 'mix_id'],
      dtype='object')

In [7]:
true_boundary_time = df.timestamp_next
true_boundary_beat = df.timestamp_beat_next

In [8]:
df['diff_out'] = (true_boundary_time - df.mix_cue_out_time).abs()
df['diff_in'] = (true_boundary_time - df.mix_cue_in_time).abs()
df['diff_mid'] = (true_boundary_time - df.mix_cue_mid_time).abs()
df['diff_best'] = df[['diff_in', 'diff_out', 'diff_mid']].T.min()

In [9]:
df['diff_out_beat'] = (true_boundary_beat - df['mix_cue_out_beat']).abs()
df['diff_in_beat'] = (true_boundary_beat - df['mix_cue_in_beat']).abs()
df['diff_mid_beat'] = (true_boundary_beat - df['mix_cue_mid_beat']).abs()
df['diff_best_beat'] = df[['diff_in_beat', 'diff_out_beat', 'diff_mid_beat']].T.min()

In [10]:
df['hit10'] = df.diff_in < 10
df['hit15'] = df.diff_in < 15
df['hit30'] = df.diff_in < 30
df['hit60'] = df.diff_in < 60
df['hit10_best'] = df['diff_best'] < 10
df['hit15_best'] = df['diff_best'] < 15
df['hit30_best'] = df['diff_best'] < 30
df['hit60_best'] = df['diff_best'] < 60

In [11]:
df['hit_beat16'] = df['diff_in_beat'] < 16
df['hit_beat32'] = df['diff_in_beat'] < 32
df['hit_beat64'] = df['diff_in_beat'] < 64
df['hit_beat128'] = df['diff_in_beat'] < 128
df['hit_beat16_best'] = df['diff_best_beat'] < 16
df['hit_beat32_best'] = df['diff_best_beat'] < 32
df['hit_beat64_best'] = df['diff_best_beat'] < 64
df['hit_beat128_best'] = df['diff_best_beat'] < 128

In [12]:
df['matched'] = (df.match_rate_prev > MATCH_RATE_THRESHOLD) & (df.match_rate_next > MATCH_RATE_THRESHOLD)
df_matched = df[df.matched]

In [13]:
df.head()

,case,i_track_prev,i_track_next,track_id_prev,track_id_next,match_rate_prev,match_rate_next,timestamp_prev,timestamp_next,mix_cue_out_time,...,hit60_best,hit_beat16,hit_beat32,hit_beat64,hit_beat128,hit_beat16_best,hit_beat32_best,hit_beat64_best,hit_beat128_best,matched
0,chroma+mfcc,0,1,0tNDAyqBm2A,yAFEClw71TM,0.819165,0.112654,0,320,283.88,...,True,False,False,False,False,False,False,False,True,False
1,chroma-keyinv,0,1,0tNDAyqBm2A,yAFEClw71TM,0.813846,0.183642,0,320,283.88,...,True,False,False,False,False,False,False,False,True,False
2,mfcc,0,1,0tNDAyqBm2A,yAFEClw71TM,0.851623,0.123664,0,320,304.06,...,True,False,False,False,False,False,True,True,True,False
3,chroma+mfcc-keyinv,0,1,0tNDAyqBm2A,yAFEClw71TM,0.819165,0.259259,0,320,283.88,...,True,False,False,False,False,False,False,False,True,False
4,chroma,0,1,0tNDAyqBm2A,yAFEClw71TM,0.813846,0.089506,0,320,283.88,...,True,False,False,False,False,False,False,False,True,False


# Statistics

In [14]:
df.drop_duplicates(['case', 'mix_id', 'i_track_prev']).groupby('case').size()

case
chroma                30
chroma+mfcc           30
chroma+mfcc-keyinv    30
chroma-keyinv         30
mfcc                  30
dtype: int64

In [15]:
df_matched.drop_duplicates(['case', 'mix_id', 'i_track_prev']).groupby('case').size()

case
chroma                16
chroma+mfcc           19
chroma+mfcc-keyinv    18
chroma-keyinv         16
mfcc                   5
dtype: int64

# Scores 

## Differences

### Median & mean differences in time

In [16]:
df_matched.groupby('case')[['diff_in','diff_out','diff_mid', 'diff_best']].agg(['median', 'mean']).round(1)

diff_in       diff_out       diff_mid       diff_best     
                    median  mean   median  mean   median  mean    median mean
case                                                                         
chroma                12.0  23.2     13.3  17.5      7.9  15.6       5.4  9.8
chroma+mfcc           13.1  28.8     12.4  22.3      6.2  13.0       3.3  9.1
chroma+mfcc-keyinv    13.0  29.5     13.1  22.9      6.7  13.5       3.4  9.5
chroma-keyinv         12.0  23.2     13.3  17.5      7.9  15.6       5.4  9.8
mfcc                   3.5  20.5     13.7  13.3      6.4  10.2       2.3  7.7

### Median & mean differences in beat

In [17]:
df_matched.groupby('case')[['diff_in_beat','diff_out_beat','diff_mid_beat', 'diff_best_beat']].agg(['median', 'mean']).round(1)

diff_in_beat       diff_out_beat       diff_mid_beat        \
                         median  mean        median  mean        median  mean   
case                                                                            
chroma                     23.5  50.2          26.0  34.5          16.0  32.2   
chroma+mfcc                27.0  60.9          26.0  47.6          12.0  26.9   
chroma+mfcc-keyinv         25.5  62.6          26.0  49.1          12.5  28.2   
chroma-keyinv              23.5  50.2          26.0  34.5          16.0  32.2   
mfcc                        7.0  54.4          26.0  27.2          13.0  24.2   

                   diff_best_beat        
                           median  mean  
case                                     
chroma                       10.5  19.1  
chroma+mfcc                   7.0  19.1  
chroma+mfcc-keyinv            7.0  19.8  
chroma-keyinv                10.5  19.1  
mfcc                          5.0  16.4

## Hit-rates

### Hit-rates in time

In [18]:
df_matched.groupby('case')[['hit10', 'hit15','hit30','hit60', 'hit10_best', 'hit15_best','hit30_best','hit60_best']].mean()

,hit10,hit15,hit30,hit60,hit10_best,hit15_best,hit30_best,hit60_best
case,,,,,,,,
chroma,0.437500,0.562500,0.750000,0.812500,0.750000,0.937500,0.937500,0.937500
chroma+mfcc,0.368421,0.526316,0.789474,0.789474,0.789474,0.894737,0.947368,0.947368
chroma+mfcc-keyinv,0.388889,0.555556,0.777778,0.777778,0.777778,0.888889,0.944444,0.944444
chroma-keyinv,0.437500,0.562500,0.750000,0.812500,0.750000,0.937500,0.937500,0.937500
mfcc,0.600000,0.600000,0.800000,0.800000,0.800000,0.800000,0.800000,1.000000


### Hit-rates in beat

In [19]:
df_matched.groupby('case')[[
'hit_beat16',
'hit_beat32',
'hit_beat64',
'hit_beat128',
'hit_beat16_best',
'hit_beat32_best',
'hit_beat64_best',
'hit_beat128_best',
]].mean()

,hit_beat16,hit_beat32,hit_beat64,hit_beat128,hit_beat16_best,hit_beat32_best,hit_beat64_best,hit_beat128_best
case,,,,,,,,
chroma,0.437500,0.625000,0.750000,0.875000,0.625000,0.937500,0.937500,1.0
chroma+mfcc,0.368421,0.578947,0.789474,0.842105,0.684211,0.842105,0.894737,1.0
chroma+mfcc-keyinv,0.388889,0.611111,0.777778,0.833333,0.666667,0.833333,0.888889,1.0
chroma-keyinv,0.437500,0.625000,0.750000,0.875000,0.625000,0.937500,0.937500,1.0
mfcc,0.600000,0.800000,0.800000,0.800000,0.800000,0.800000,0.800000,1.0
